In [1]:
from scipy.io import loadmat
import numpy as np
from numpy import *
import pandas as pd

In [2]:
# 从abcd_rfMRI_timeseries_HCPMMP.mat 中截取的前20条样例数据
ts = loadmat('/Users/eugene_zh/Documents/MATLAB/sample20.mat')


In [3]:
ts['sample20'][0]

array([array([[  -4.37374449,  -22.32160187,  -20.69812775, ...,   -9.39270115,
                 19.38031006,    5.37024879],
              [  -4.62811375,  -23.56358337,  -21.99608612, ...,   -9.88006878,
                 19.80414391,    6.40895939],
              [  -4.86236191,  -24.75100327,  -23.31113815, ...,  -10.32406235,
                 19.90430641,    7.74595642],
              ...,
              [ -34.59323502,  -97.56585693, -111.02809906, ...,   52.52920914,
                 65.79203796,   63.20910645],
              [ -28.63887596,  -82.40544891,  -92.78044891, ...,   43.44189072,
                 56.26488113,   53.96816635],
              [ -20.56902504,  -65.36425781,  -72.43799591, ...,   31.01912308,
                 44.4006424 ,   37.5317421 ]])                                 ,
       array([[  2.58531117,   1.01859999,  -8.02578545, ...,  -8.47011471,
                -0.43093589,  -4.25915432],
              [  2.64434004,   1.08936942,  -8.12581062, ...,  -9.0242

In [4]:
np.shape(ts['sample20'][0])

(2,)

In [5]:
ts['sample20'][0][0]

array([[  -4.37374449,  -22.32160187,  -20.69812775, ...,   -9.39270115,
          19.38031006,    5.37024879],
       [  -4.62811375,  -23.56358337,  -21.99608612, ...,   -9.88006878,
          19.80414391,    6.40895939],
       [  -4.86236191,  -24.75100327,  -23.31113815, ...,  -10.32406235,
          19.90430641,    7.74595642],
       ...,
       [ -34.59323502,  -97.56585693, -111.02809906, ...,   52.52920914,
          65.79203796,   63.20910645],
       [ -28.63887596,  -82.40544891,  -92.78044891, ...,   43.44189072,
          56.26488113,   53.96816635],
       [ -20.56902504,  -65.36425781,  -72.43799591, ...,   31.01912308,
          44.4006424 ,   37.5317421 ]])

In [7]:
for i in range(1,20):
    print(np.shape(ts['sample20'][i][0]))
    print(np.shape(ts['sample20'][i][1]))

(1520, 360)
(1520, 21)
(1532, 360)
(1532, 21)
(1532, 360)
(1532, 21)
(1532, 360)
(1532, 21)
(1532, 360)
(1532, 21)
(1532, 360)
(1532, 21)
(1532, 360)
(1532, 21)
(1532, 360)
(1532, 21)
(1149, 360)
(1149, 21)
(1520, 360)
(1520, 21)
(1149, 360)
(1149, 21)
(1395, 360)
(1395, 21)
(1149, 360)
(1149, 21)
(1520, 360)
(1520, 21)
(1532, 360)
(1532, 21)
(1149, 360)
(1149, 21)
(1520, 360)
(1520, 21)
(1532, 360)
(1532, 21)
(1532, 360)
(1532, 21)


每个被试有两个矩阵，第一个（1532，360）第一维是时间，第二维是脑区（HCPMMP模版），第二个矩阵（1532，21）是另一个脑区模版AESG，freesurfer。

In [4]:
samp1 = ts['sample20'][1]

In [30]:
samp1[1][:,1][0:11]

array([-4.90239143, -5.2858119 , -5.70264292, -6.22754908, -6.94361925,
       -7.9502697 , -8.97879124, -9.43782711, -9.22953987, -8.7144804 ,
       -8.23374748])

In [28]:
samp1[1][:,1][10:20]

array([-8.23374748, -7.74812269, -6.97687101, -5.9221983 , -4.72674561,
       -3.70404243, -3.09830952, -2.74183249, -2.47017074, -2.2878406 ])

单个节点时序可变性，反应单个脑区的时间稳定性

In [43]:
def fc_vari(data,roi,window,*,stride = 0):
    """
    ***********parameters*************
    data: timeseries of fMRI,each columns match a specific region
    roi: number of region of interest
    window: window length
    stride: step of each move
    """
    
    if not stride: # stride为0 无重叠滑窗
        stride = window
    i = 0
    corlist = []
    while (i+window) < len(data)-1:
        cor = np.corrcoef(data[:,roi][i:i+window],data[:,roi][i+window:i+window*2])
        corlist.append(cor)
        i += stride
    Vk = 1-np.mean(corlist)
    return Vk

In [41]:
def fc_var_all(data,window,*,stride = 0):
    if not stride: # stride为0 无重叠滑窗
        stride = window
    i = 0
    corlist = []
    while (i+window) < len(data)-1:
        cor = np.corrcoef(data[i:i+window],data[i+window:i+window*2])
        corlist.append(cor)
        i += stride
    Vk = 1-mean(corlist)
    return Vk

In [44]:
Vk1_1 = fc_vari(samp1[1],0,10)

这个是比较标准的代码

In [45]:
def fc_var_all(data,window,*,stride = 0,roi = 0):
    """
    ***********parameters*************
    data: timeseries of fMRI,each columns match a specific region
    
    window: window length
    stride: step of each move
    roi: number of region of interest,start from 1, if 0 is received, return all
    **********************************
    """
    
    
    if not stride: # stride为0 无重叠滑窗
        stride = window
        
    # 计算每个窗口下的FC矩阵
    i = 0
    fclist=[] # 每个窗口下的FC矩阵列表
    while (i+window) < len(data)-1:
        fc = np.corrcoef(data[i:i+window],rowvar=False)
        fclist.append(fc)
        i += stride
    
    # 计算动态变化性
    corlist=[] # 每个脑区的列表
    # 对每个脑区循环
    for k in range(0,data.shape[1]):
        corarray = []
        for i in range(1,len(fclist)):
            for j in range(0,i):
                cor = cov(fclist[i][k],fclist[j][k])/(std(fclist[i][k])*std(fclist[j][k]))
                # coarray--(1*(n(n-1)/2),n=window length)
                corarray.append(cor)
        Vk = 1-mean(corarray)
        corlist.append(Vk)
    
    if roi:
        return corlist[roi-1]
    else:
        return corlist

In [49]:
Vk0 = fc_var_all(samp1[1],100,roi=1)
Vk0

0.19565045491216626

In [44]:
Vk

[0.19565045491216626,
 0.22516651558081147,
 0.24600618524736428,
 0.276842344968519,
 0.2360624279876068,
 0.2654266391608099,
 0.2255937885295003,
 0.23188514981805908,
 0.22188386815123418,
 0.16448967190264918,
 0.17501598352683023,
 0.2706376555184301,
 0.2520849778595837,
 0.23017007381331156,
 0.23716442527008086,
 0.2363117212873448,
 0.23260387887649558,
 0.17036270888122196,
 0.18093088989927764,
 0.27419890656384904,
 0.2112614108503189]

In [29]:
pd.DataFrame(a)

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
0,1.000000,-0.232294,-0.159754,0.001978,0.123252,-0.095666,-0.334438,0.086394,-0.166355,-0.702422,...,0.058124,-0.129767,0.034137,-0.057487,0.046768,-0.225348,0.023780,-0.028019,-0.075713,-0.200882
1,-0.232294,1.000000,0.090330,0.026685,-0.144543,-0.042981,-0.345379,-0.225857,0.060532,-0.224438,...,-0.298612,-0.162900,-0.096732,0.114014,-0.086916,0.187378,0.017755,0.119664,-0.020181,0.164119
2,-0.159754,0.090330,1.000000,0.393515,-0.176750,-0.025035,-0.111479,0.093506,0.093137,-0.055099,...,0.230272,0.245805,0.146806,0.040539,0.248087,0.101181,0.120014,0.015702,0.210024,0.208503
3,0.001978,0.026685,0.393515,1.000000,-0.190460,-0.222082,-0.182520,0.009295,-0.147240,-0.152634,...,0.208921,0.270696,-0.026140,-0.053708,0.488818,-0.059612,0.123557,-0.002293,0.224142,0.108786
4,0.123252,-0.144543,-0.176750,-0.190460,1.000000,0.517797,0.277399,0.242228,0.195708,-0.202545,...,0.110759,-0.009962,0.155233,0.294797,-0.026969,0.077222,-0.127546,0.042254,0.112160,0.061407
5,-0.095666,-0.042981,-0.025035,-0.222082,0.517797,1.000000,0.053334,0.029939,0.131609,0.028096,...,0.125144,0.160271,0.365620,0.466738,-0.179470,0.167094,0.037690,0.173074,-0.016140,0.157433
6,-0.334438,-0.345379,-0.111479,-0.182520,0.277399,0.053334,1.000000,0.095268,0.040639,0.276186,...,-0.161571,-0.105835,-0.068224,0.033699,-0.061533,0.031576,-0.189265,-0.136891,0.038011,-0.092751
7,0.086394,-0.225857,0.093506,0.009295,0.242228,0.029939,0.095268,1.000000,0.688030,-0.237633,...,0.242033,0.128648,0.058193,-0.064890,0.118563,0.042621,0.201841,0.195415,-0.005027,0.122313
8,-0.166355,0.060532,0.093137,-0.147240,0.195708,0.131609,0.040639,0.688030,1.000000,-0.070871,...,0.158134,0.019778,0.199568,0.113577,-0.091472,0.132204,0.220968,0.248556,-0.064166,0.146055
9,-0.702422,-0.224438,-0.055099,-0.152634,-0.202545,0.028096,0.276186,-0.237633,-0.070871,1.000000,...,-0.060987,0.046812,-0.091453,-0.050193,-0.217504,-0.024125,-0.233187,-0.227187,-0.139920,-0.134992


In [30]:
a[:,2]

array([-0.15975422,  0.0903302 ,  1.        ,  0.39351497, -0.17675005,
       -0.02503471, -0.11147909,  0.09350641,  0.09313662, -0.05509914,
       -0.03541157,  0.23027177,  0.2458049 ,  0.14680644,  0.04053862,
        0.24808691,  0.10118092,  0.12001353,  0.01570179,  0.21002424,
        0.20850323])

In [24]:
apd.DataFrame(samp1[1]).corr()

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
0,1.000000,-0.232294,-0.159754,0.001978,0.123252,-0.095666,-0.334438,0.086394,-0.166355,-0.702422,...,0.058124,-0.129767,0.034137,-0.057487,0.046768,-0.225348,0.023780,-0.028019,-0.075713,-0.200882
1,-0.232294,1.000000,0.090330,0.026685,-0.144543,-0.042981,-0.345379,-0.225857,0.060532,-0.224438,...,-0.298612,-0.162900,-0.096732,0.114014,-0.086916,0.187378,0.017755,0.119664,-0.020181,0.164119
2,-0.159754,0.090330,1.000000,0.393515,-0.176750,-0.025035,-0.111479,0.093506,0.093137,-0.055099,...,0.230272,0.245805,0.146806,0.040539,0.248087,0.101181,0.120014,0.015702,0.210024,0.208503
3,0.001978,0.026685,0.393515,1.000000,-0.190460,-0.222082,-0.182520,0.009295,-0.147240,-0.152634,...,0.208921,0.270696,-0.026140,-0.053708,0.488818,-0.059612,0.123557,-0.002293,0.224142,0.108786
4,0.123252,-0.144543,-0.176750,-0.190460,1.000000,0.517797,0.277399,0.242228,0.195708,-0.202545,...,0.110759,-0.009962,0.155233,0.294797,-0.026969,0.077222,-0.127546,0.042254,0.112160,0.061407
5,-0.095666,-0.042981,-0.025035,-0.222082,0.517797,1.000000,0.053334,0.029939,0.131609,0.028096,...,0.125144,0.160271,0.365620,0.466738,-0.179470,0.167094,0.037690,0.173074,-0.016140,0.157433
6,-0.334438,-0.345379,-0.111479,-0.182520,0.277399,0.053334,1.000000,0.095268,0.040639,0.276186,...,-0.161571,-0.105835,-0.068224,0.033699,-0.061533,0.031576,-0.189265,-0.136891,0.038011,-0.092751
7,0.086394,-0.225857,0.093506,0.009295,0.242228,0.029939,0.095268,1.000000,0.688030,-0.237633,...,0.242033,0.128648,0.058193,-0.064890,0.118563,0.042621,0.201841,0.195415,-0.005027,0.122313
8,-0.166355,0.060532,0.093137,-0.147240,0.195708,0.131609,0.040639,0.688030,1.000000,-0.070871,...,0.158134,0.019778,0.199568,0.113577,-0.091472,0.132204,0.220968,0.248556,-0.064166,0.146055
9,-0.702422,-0.224438,-0.055099,-0.152634,-0.202545,0.028096,0.276186,-0.237633,-0.070871,1.000000,...,-0.060987,0.046812,-0.091453,-0.050193,-0.217504,-0.024125,-0.233187,-0.227187,-0.139920,-0.134992


In [55]:
pd.DataFrame(samp1[0]).corr()

,0,1,2,3,4,5,6,7,8,9,...,350,351,352,353,354,355,356,357,358,359
0,1.000000,-0.002004,0.552953,0.947102,0.774435,0.681906,0.798055,-0.174108,-0.020013,0.002339,...,-0.039720,-0.174078,0.122786,0.146960,-0.005238,-0.107383,-0.019649,0.031655,0.145414,-0.158821
1,-0.002004,1.000000,0.109056,0.085815,0.123617,0.306334,0.235273,-0.023192,-0.069527,0.100435,...,-0.301964,-0.164305,-0.097687,-0.198590,-0.129557,-0.114474,0.080113,-0.088101,-0.072401,0.115729
2,0.552953,0.109056,1.000000,0.582349,0.294705,0.334722,0.395011,-0.180883,-0.051813,0.102160,...,-0.181817,-0.018176,0.018772,0.071925,-0.069061,-0.068511,-0.041572,0.063317,0.026923,-0.108880
3,0.947102,0.085815,0.582349,1.000000,0.842816,0.784533,0.874143,-0.184914,-0.037448,0.047890,...,-0.113413,-0.110703,0.100086,0.078940,-0.047062,-0.115047,-0.057791,-0.041355,0.147209,-0.110433
4,0.774435,0.123617,0.294705,0.842816,1.000000,0.900788,0.840088,-0.158735,-0.082610,-0.037110,...,-0.097111,-0.091670,0.070200,-0.004768,-0.034744,-0.004595,0.018191,-0.083709,0.142479,-0.075540
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
355,-0.107383,-0.114474,-0.068511,-0.115047,-0.004595,-0.073487,-0.204112,-0.085877,0.068442,-0.428948,...,0.014306,0.243853,-0.106744,-0.114779,-0.175826,1.000000,0.246653,-0.115506,-0.065162,0.253573
356,-0.019649,0.080113,-0.041572,-0.057791,0.018191,-0.032251,-0.098363,-0.112428,-0.134944,-0.178278,...,0.080589,-0.380823,-0.177109,-0.159852,-0.027878,0.246653,1.000000,-0.098532,0.111593,0.215009
357,0.031655,-0.088101,0.063317,-0.041355,-0.083709,-0.026444,0.070319,0.392528,0.229455,0.308862,...,0.323075,0.013956,0.649201,0.663234,0.641077,-0.115506,-0.098532,1.000000,0.097700,-0.038738
358,0.145414,-0.072401,0.026923,0.147209,0.142479,0.150312,0.160796,-0.235660,-0.140074,-0.162476,...,-0.073486,-0.180817,-0.022719,0.121104,0.052967,-0.065162,0.111593,0.097700,1.000000,0.506089


In [57]:
np.corrcoef(samp1[0],rowvar=False).shape

(360, 360)